<div style="background-image: linear-gradient(145deg, rgba(35, 47, 62, 1) 0%, rgba(0, 49, 129, 1) 40%, rgba(32, 116, 213, 1) 60%, rgba(244, 110, 197, 1) 85%, rgba(255, 173, 151, 1) 100%); padding: 1rem 2rem; width: 95%"><img style="width: 60%;" src="../../images/MLU_logo.png"></div>

# <a name="0">MLU Mathematical Fundamentals for Machine Learning</a>
# <a name="0">Lecture 4: Differential calculus</a>
## <a name="0">Lab 4.2: Gradient descent</a>

* <a href="#0">Maximum Likelihood Estimation (MLE)</a> 
* <a href="#1">Bernoulli probability estimation</a> 
* <a href="#2">Implementation of Gradient Descent in one dimension</a> 
* <a href="#3">Multivariate Gradient Descent</a>   


[**Gradient descent**](https://en.wikipedia.org/wiki/Gradient_descent) is a fundamental optimization algorithm at the heart of many modern machine learning techniques. This iterative approach finds the minimum of a function by following the direction of steepest descent along its surface, much like a ball rolling down a hill would naturally find the lowest point. While simple in concept, gradient descent's effectiveness and versatility have made it an indispensable tool in data science and optimization problems.

In this lab, you will implement gradient descent from scratch to develop an understanding of its mechanics. You will explore how the algorithm navigates different types of functions, observe its convergence behavior, and investigate the impact of different learning rates and initial conditions. You will start by applying it to a single-variable optimization problem of known solution, so that you can compare how gradient descent compares to the exact result. Then you will extend the approach to multi-variable functions. 



In [ ]:
# Upgrade libraries
!pip install -q --upgrade pip
!pip install -q --upgrade scikit-learn

In [ ]:
%%capture
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import torch

from collections import Counter
from matplotlib import cm

from IPython.display import Markdown, display

# Set a seed for reproducibility
np.random.seed(99)

%matplotlib inline

## <a name="0">Maximum Likelihood Estimation (MLE)</a> 
(<a href="#0">Go to top</a>)

[**Maximum Likelihood Estimation (MLE)**](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation) is a method to estimate the parameters of an assumed probability distribution, given some observed data. This is achieved by maximizing a likelihood function so that, under the assumed statistical model, the observed data is most probable. 

__MLE Step 1__
 * Propose a __model__ (with its parameters) to capture the underlying relationship in the dataset

__MLE Step 2__
 * Compute the __Likelihood__ (the joint probability) that this model generates the data (a function of the model parameters). The goal is to maximize the Likelihood

__MLE Step 3__
 * Compute the negative log-Likelihood (__Loss function__). The goal is to minimize the Loss function, $\cal{L}({p})$, where ${p}$ represents the model parameters. Find the model that __minimizes the Loss function__, which is equivalent to maximizing the Likelihood

__MLE Step 4__
 * Take the __derivative of the Loss function, set it to zero__

__MLE Step 5__ 
 * __Solve__ for the model parameters that minimize the Loss function, i.e. maximize the Likelihood

__Gradient Descent__ 

In cases where the derivative can't be solved analytically, numerical methods like gradient descent are needed. Recall that the gradient descent approach to approximating the best model parameters ${p}$ to minimize the loss function implies the following:

 * Start with an initial guess of $p_0$, and a learning rate $\eta$.
 * Iterate for a number of iterations, updating the parameters with the negative gradient of the loss function
 $$p_{i+1} = p_i - \eta * \frac{\partial \cal{L}}{\partial p_i}$$ 

## <a name="1">Bernoulli probability estimation</a> 
(<a href="#0">Go to top</a>)

A **Bernoulli trial** represents an experiment with exactly two possible outcomes: success or failure. These outcomes are: 

* mutually exclusive and exhaustive, meaning only one can occur at a time, and 
* together they represent all possible outcomes. 

Common examples include flipping a coin (heads or tails), testing whether a manufactured part is defective (pass or fail), or checking if a customer makes a purchase (yes or no).

In a sequence of Bernoulli trials, each trial is independent of the others, and the probability of success ($p$) remains constant throughout all trials. This probability is often unknown in real-world applications, and estimating it accurately becomes crucial for many practical applications, from quality control in manufacturing to predicting customer behavior in marketing.

Maximum Likelihood Estimation (MLE) provides a method for estimating this unknown probability parameter based on observed data from a series of Bernoulli trials. The likelihood function for a set of $n$ binary 0/1 observations $X = x_1, ..., x_n$, of which exactly $k$ are a success one can be written as:
$$
L(p; X) = p^k (1-p)^{n-k}
$$

To estimate $p$ MLE instructs to minimize the negative logarithm of said likelihood, that can be written as:
$$
\mathcal{L}(p; X) = - k \log(p) - (n-k) \log(1-p)
$$


While this optimization problem has a closed-form solution for Bernoulli trials, given by $\hat{p} = k/n$, we will solve it using gradient descent to illustrate a more general approach that can be applied to more complex probability distributions. Gradient descent iteratively adjusts our estimate of $p$ by following the direction of steepest increase in the likelihood function (or, equivalently, the log-likelihood function), eventually converging to the maximum likelihood estimate. 


### Generate data

Let's generate data for $n$ Bernoulli experiments by sampling from the binomial distribution, of which the Bernoulli distribution is a special case. In `NumPy` the implementation is `np.random.binomial(size=n_observations, p=p, n=1)`, i.e. setting `n=1` in `np.random.binomial` gives us a Bernoulli trial. See [here](https://math.stackexchange.com/questions/838107/what-is-the-difference-and-relationship-between-the-binomial-and-bernoulli-distr) for more information. 

We will generate 200 data points that could represent Amazon customers making, or not, a purchase. This is parameter `size` in the `binomial` below. We'll take a value for the probability $p$ of purchasing close to 18%. These parameters can be adjusted in the code.  

In [ ]:
data = np.random.binomial(size=200, p=0.182, n=1)

The MLE estimate for $p$ given this data is simply $k/n$ where $k$ is the number of successes and $n$ the total number of experiments. This is the true estimate, as it's found analytically by solving of the negative log likelihood minimization as you saw in the Lecture. 

In [ ]:
c = Counter(data)
n_successes = c[1]
n_observations = len(data)

true_p = n_successes/n_observations
print(f"Estimated (true) p = {true_p}")

Let's plot the shape of the likelihood and the loss (negative log-likelihood) as a function of the parameter $p$, given by the formulas $L(p; X) = p^k (1-p)^{n-k}$ and $Loss(p; X) = - k \log(p) - (n-k) \log(1-p)$. 

We will use `torch` tensors in the following, as we'll be reusing code for the gradient descent computation.

In [ ]:
def likelihood(p, n, k):
    return p ** k * (1 - p) ** (n - k)

def loss_bernoulli(p, n, k):
    return - k * torch.log(p) - (n - k) * torch.log(1 - p)

In [ ]:
p_range = torch.arange(0, 1, 0.001)

plt.figure(figsize=(6, 4))
plt.plot(p_range, likelihood(p_range, n_observations, n_successes))
plt.axvline(true_p, color='g', linestyle='dashed')
plt.xticks([0, true_p, 0.5, 0.75, 1])
plt.xlabel("p")
plt.ylabel("Likelihood")
plt.title("Likelihood Function")
plt.show()

plt.figure(figsize=(6, 4))
plt.plot(p_range, loss_bernoulli(p_range, n_observations, n_successes))
plt.axvline(true_p, color='g', linestyle='dashed')
plt.xticks([0, true_p, 0.5, 0.75, 1])
plt.xlabel("p")
plt.ylabel("Loss")
plt.title("Loss Function")
plt.show()

Although it's difficult to appreciate with these particular parameters, the peak (maximum) of the likelihood occurs at the same $p$ that the minimum of the loss function and happens at exactly `n_successes`/`n_observations` for the observed dataset. That's indicated by the dashed vertical line.

### Exercise 1

<div style="align: left; border: 4px solid cornflowerblue; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 80%; max-height:80%; margin: 5px;" src="../../images/MLU_challenge.png" alt="MLU challenge" width=12% height=12%/>
    <span style="padding: 20px; align: left;">
        <p><b>It is your turn!</b></p>
        <p><b>Exercise 1. Visualize the Bernoulli distribution.</b></p>
        <p>Try generating different data for the Bernoulli experiments and plot the resulting likelihood and loss function. You can play around with the value of the probability parameter <code>p</code> and also with the number of datapoints <code>size</code>.</p>
        <p>Notice that for a large number of observations <code>n</code> you might get a numerical error due to the small values in the likelihood function, since it's the product of many numbers smaller than one. This gets worse when the number of data points increases. The loss function is more robust as it changes the calculation from the product of probabilities to the sum of log-probabilities.</p>
        </span>
</div>

In [ ]:
###### YOUR CODE HERE ######






###### END OF CODE ######

## <a name="2">Implementation of Gradient Descent in one dimension</a> 
(<a href="#0">Go to top</a>)

### Gradient Descent

While the loss function above is simple enough that we can compute its derivative and solve for the value where the derivative is 0, here we discuss how to approach this numerically in Python with gradient descent and `autograd` in `torch` for computing those gradients automatically.

#### Initialization of parameters

We start by initializing all parameters that we want to estimate, in this case it's only the probability $p$. It's convenient to use a random initialization between 0 and 1, as we know the probability must lie in this range. 

In [ ]:
# Initialize p randomly between 0 and 1
p = torch.rand(1, requires_grad=True)

#### Learning rate

We need to choose a learning rate that determines how big of a step we take in the direction of the negative gradient. This is a hyperparameter that often requires some tuning. Typically, it's a value that might range bewtween $10^{-5}$ and $10^{-1}$, but it depends on the actual problem and architecture.

In [ ]:
# Set learning rate
learning_rate = 0.0001

#### Gradient Descent Loop
Now we can implement the gradient descent algorithm:

1. Compute the loss
2. Compute the gradient of the loss with respect to its parameters
3. Update the parameters by taking a step in the direction of the negative gradient
4. Repeat until convergence. You can keep track of the loss to visualize the training progress

Here's how we can implement this in PyTorch:

In [ ]:
# Let's generate new data
data = np.random.binomial(size=200, p=0.182, n=1)
c = Counter(data)
n_successes = c[1]
n_observations = len(data)
print(f"Estimated (true) p = {n_successes/n_observations}")

In [ ]:
# Set the number of iterations. This is called the number of epochs
num_iterations = 100

# Store train losses for visualization
train_losses = []

# Perform Gradient Descent
for i in range(num_iterations):
    # Compute the loss
    loss = loss_bernoulli(p, n_observations, n_successes)

    # Compute gradients
    loss.backward()

    # Update parameters
    # Use the .no_grad() context manager to temporarily disable gradient computation
    # We don't want pytorch to create a computational graph of the subtraction operation - it's only a parameter update
    with torch.no_grad():
        p -= learning_rate * p.grad

    # Zero gradients for next iteration
    # This is needed, otherwise pytorch accumulates the gradient in .grad - that's not what we want
    p.grad.zero_()

    train_loss = loss
    train_losses.append(train_loss.detach().numpy())

    # Optional: Print loss every 10 iterations
    # This is good to check that the loss is decreasing
    if i % 10 == 0:
        print(f'Iteration {i}, Loss: {loss.item():.4f}, p: {p.item():.4f}')

#### Results

When the training is finished, the variable `p` contains the value found by gradient descent. We can first visualize the training loss to ensure that its value has decreased during training. 

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(range(num_iterations), train_losses)
plt.xlabel("Number of iterations")
plt.ylabel("Train Loss")
plt.title("Train Loss Evolution")
plt.show()

Once the value of the training loss reaches a plateau, we consider that the algorithm has found a local minimum of the loss function. In this case the Bernoulli loss only has one minimum. Let's now see how well gradient descent has approximated the true value of the estimated parameter $p$.

In [ ]:
print(f"Estimated (true) p = {n_successes/n_observations}")

print(f"Estimated parameter value, after {num_iterations} steps of gradient descent = {p.data.item()}")

## <a name="3">Multivariate Gradient Descent</a> 
(<a href="#0">Go to top</a>)

While the previous section focused on optimizing a single parameter, many real-world problems involve multiple parameters. Multivariate gradient descent extends the concept to optimize functions with multiple variables simultaneously.

In multivariate gradient descent, instead of dealing with a single gradient, we work with a gradient vector. Each element of this vector represents the partial derivative of the loss function with respect to one of the parameters.

To illustrate this, let's pick a function of two parameters:

$$
f(x_1, x_2) = (x_1^{2}+x_2-11)^{2}+(x_1+x_2^{2}-7)^{2}
$$

This is the [**Himmelblau's function**](https://en.wikipedia.org/wiki/Himmelblau%27s_function). This function has four identical local minima at:

* $P_1 = (3.0, 2.0)$
* $P_2 = (-2.805118, 3.131312)$
* $P_3 = (-3.779310, -3.283186)$
* $P_4 = (3.584428, -1.848126)$

It is typically evaluated on the square domain $-5 ≤ x_1, x_2 ≤ 5$. The Himmelblau function is particularly useful in demonstrating how different optimization algorithms behave in the presence of multiple optima. Depending on the starting point and the specific algorithm used, an optimization process might converge to different minima.

When visualized in 3D, it shows a distinctive pattern of four valleys where the minima are located, separated by ridges. 

In [ ]:
# Himmelblau's function: four zeros/mins (3, 2), (-2.8,3.13) (-3.78,-3.28), (3.58,-1.84)
def loss_2d(X):
    x, y = X[0], X[1]
    return (x ** 2 + y - 11) ** 2 + (x + y ** 2 - 7) ** 2

# Make data 
X1 = np.arange(-5, 5, 0.15)
X2 = np.arange(-5, 5, 0.15)
X1, X2 = np.meshgrid(X1, X2)
X = [X1, X2]

Z = loss_2d(X)

# Plot the surface.
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(8,8))
surf = ax.plot_surface(X1, X2, Z, 
                       cmap=cm.RdYlBu,
                       linewidth=0, 
                       alpha=0.75)

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.7, aspect=8)

plt.show()

### Exercise 2

<div style="align: left; border: 4px solid cornflowerblue; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 80%; max-height:80%; margin: 5px;" src="../../images/MLU_challenge.png" alt="MLU challenge" width=12% height=12%/>
    <span style="padding: 20px; align: left;">
        <p><b>It is your turn!</b></p>
        <p><b>Exercise 2. Implement gradient descent in two dimensions to minimize Himmelblau's function.</b></p>
        <p>This time you will need to define and initialize 2 parameters, x1 and x2. Instead of initializing them in the interval (0, 1), you can choose an alternative recommended initialization: a sample from a normal distribution of mean 0 and standard deviation 1. The loss function will be the Himmelsblau function defined above.</p>
        <p>You can use the following training parameters to ensure that your training runs: a learning rate of 0.001 and 300 iterations. But you can also experiment with other values.</p>
        <p>When you're done training, plot the train loss. Is the training working?</p>
        <p>Inspect the value of the minimum that you have found. Is it one of the 4 minima of the function?</p>
        <p>Run the training several times, each with a different random initialization. Do you always end up finding the same minimum?</p>
        </span>
</div>

In [ ]:
###### YOUR CODE HERE ######






###### END OF CODE ######

<div style="align: left; border: 4px solid lightcoral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="../../images/MLU_question.png" alt="MLU solution" width=12% height=12%/>
    <span style="padding: 20px; align: left;">
        <p><b>Challenge Help</b></p>
        <p>You can mostly reuse the code from the 1-dimensional gradient descent above, but you will need to define and update two parameters this time.</p>
        <p>If you're stuck, remove the <code>#</code> before the <code>load</code> instruction in the next code cell to display a sample solution.</p>
    </span>
</div>

In [ ]:
# %load solutions/lab42_ex2_solutions.txt

#### Conclusion

This lab explicitly shows how to implement gradient descent from scratch to minimize functions of one and multiple parameters. The exercises above should have given you enough practice to be able to implement gradient descend in the Final Project. 

<div style="display: flex; align-items: center; justify-content: left; background-color:#330066; width:99%;"> 
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="../../images/MLU_robot.png" alt="MLU robot" width="100" height="100"/>
    <span style="color: white; padding-left: 10px; align: left; margin: 15px;">
        <h3>Congratulations!</h3>
        You have completed Lab 4.2: Gradient Descent of Lecture 4: Differential Calculus of MLU Mathematical Fundamentals of Machine Learning.
        <br/>
    </span>
</div>